In [1]:
%load_ext autoreload
%autoreload 2
import sys, codecs, json, os
import ttools
from twython import TwythonStreamer, Twython
from datetime import datetime
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse as parse_date

### Master loops to step thru all preprocessed data and append:
`tweet_id`,`tweet_lang`,`user_name`,`user_screen`,`user_lang`
### based on matching three columns believed to uniquely identify a tweet:
`date`,`text`,`user_id`<br>
### and filtered for faster processing on [heavily based on how preprocessed *csv files were portioned out]:
`user_followers_count`

In [2]:
%%time
BASENAMES = ['50M_250_1000_','50M_1001_5000_']
BASEFILTERS = [(250,1001),(1001,5000)]
for BASENUM in range(len(BASENAMES)):
    print('working on subset files: %s'%(BASENAMES[BASENUM]))
    
    #load in our database of all tweets for relevant users, with new attributes we will append
    datas = []
    USECOLS = ['tweet_id','tweet_lang','date','text','user_id','user_name','user_screen','user_lang','user_followers_count']
    for WAVENUM in range(1,11):
        path = '/Users/olderhorselover/USC/fall2018/csci599/teamrepo/social-swear/misc/tweets_5mAllAttributesFINAL_and_zip/tweets-5M_all_wave%s.csv'%(WAVENUM)
        iter_data = pd.read_csv(path,usecols=USECOLS,index_col=None, lineterminator='\n',iterator=True, chunksize=100000)
        data = pd.concat([chunk[(chunk['user_followers_count'] >= BASEFILTERS[BASENUM][0]) & (chunk['user_followers_count'] < BASEFILTERS[BASENUM][1])].drop(labels='user_followers_count',axis=1) for chunk in iter_data])
    #     data = data.drop(labels='user_followers_count',axis=1)
        datas.append(data)
        del iter_data
        del data
    alldatas = pd.concat(datas)
    del datas
    alldatas.shape
    print(f'Raw fullset data shape: {alldatas.shape}')
    print(f'Raw fullset data columns:\n{alldatas.columns}')
    print(f'Raw fullset data size: {alldatas.memory_usage().sum()/1000000:.2f} MB')
    
    ss = lambda x: x.replace('\r','')
    dd = lambda x: parse_date(x).strftime('%a %b %d %H:%M:%S %z %Y')
    for file in os.listdir('/Users/olderhorselover/USC/fall2018/csci599/project/processedData/50m_all_updated/'):
        startTime = time()
        if not file.startswith(BASENAMES[BASENUM]):
            continue  #do not process file
        print('working on file: %s'%file)
        path = '/Users/olderhorselover/USC/fall2018/csci599/project/processedData/50m_all_updated/%s'%(file)
        df = pd.read_csv(path,index_col=None, header=0, lineterminator='\n',converters={'text':ss,'date':dd})
        try:
            df = df.drop(labels='date1\r',axis=1)
        except:
            pass
        print(f'raw processed data shape: {df.shape}')
        print(f'raw processed data columns:\n{df.columns}')
        print(f'Raw processed data size: {df.memory_usage().sum()/1000000:.2f} MB')
        print('PERFROMING THE MERGE!')
        dfnew = df.merge(alldatas[['user_id','text','date','tweet_id','tweet_lang','user_name','user_screen']],on=['user_id','text','date'],how='left')  # new w/ new method...also drop the 'right' columns 
        print(f'orig dataframe shape: {df.shape}')
        print(f'new dataframe shape: {dfnew.shape}')
        print(f'perc diff shape: {dfnew.shape[0]/df.shape[0]:.3f}')
        naStats = dfnew.isna().sum()
        print(f'num of tweets to drop: {naStats.tweet_id}')
        print(f'new dataframe isna() stats:\n{naStats}')
        try:
            dfnew = dfnew.dropna(axis=0,how='any',subset=['tweet_id'])  #drop any tweets that didn't get a tweet_id
        except:
            pass
        dup = dfnew.duplicated(subset=['tweet_id'])
        if ((dup.sum()/dfnew.shape[0]) > 0.07):
            print('WARNING: found a large number of duplicates in processing file: %s'%file)
        else:
            print('Non-significant number of duplicate tweets')
        print(f'Dropping duplicate tweets [tweet_id]')
        dfnew = dfnew.drop_duplicates(subset=['tweet_id'])
        print(f'FINAL dataframe shape: {dfnew.shape}')
        print('Writing dataframe to file!')
        outname = file.replace('_updated.csv','_final')
        dfnew.to_csv('./PROCESSED_FINAL_11152018/%s.csv'%(outname),index=False)
        dfnew.to_pickle('./PROCESSED_FINAL_11152018/%s.pkl'%(outname))
        del df
        del dfnew
        endTime = time()
        print('loop time for file %s'%(file))
        print(f'{(endTime - startTime)/60:0.2f} minutes\n\n\n')

    del alldatas  # no longer need/will update on next iter of BASENUM for applicable users

working on subset files: 50M_250_1000_
Raw fullset data shape: (19528151, 8)
Raw fullset data columns:
Index(['tweet_id', 'date', 'tweet_lang', 'text', 'user_id', 'user_name',
       'user_screen', 'user_lang'],
      dtype='object')
Raw fullset data size: 1406.03 MB
working on file: 50M_250_1000_9_updated.csv


<string>:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.


raw processed data shape: (1693712, 54)
raw processed data columns:
Index(['user_id', 'text', 'n_retweets', 'n_favorites', 'n_user_followers',
       'n_user_following', 'n_user_posts', 'n_user_favs', 'n_user_lists',
       'user_description_text', 'user_location', 'account_age',
       'user_created_year', 'user_created_month', 'user_geo_enabled',
       'user_img_url', 'user_banner_url', 'tweet_truncated', 'tweet_source',
       'tweet_coord', 'tweet_place', 'numMentions', 'date', 'engagment',
       'is_reply', 'is_user_bot', 'is_outlier', 'is_valid', 'eggplant',
       'mfinger', 'curse', 'fuck', 'text_polarity', 'text_subjectivity',
       'vader_comp', 'vader_pos', 'vader_neu', 'vader_neg', 'processed_text',
       'text_token', 'text_token_rem_stop', 'text_len', 'n_mentions',
       'n_hashtags', 'n_links', 'n_emojis', 'swear_count', 'swear_present',
       'swear_severity', 'swear_rarity_by_percentage', 'censored_presence',
       'day_of_week', 'hour_of_day', 'per_engagement']

<string>:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.


raw processed data shape: (1716852, 54)
raw processed data columns:
Index(['user_id', 'text', 'n_retweets', 'n_favorites', 'n_user_followers',
       'n_user_following', 'n_user_posts', 'n_user_favs', 'n_user_lists',
       'user_description_text', 'user_location', 'account_age',
       'user_created_year', 'user_created_month', 'user_geo_enabled',
       'user_img_url', 'user_banner_url', 'tweet_truncated', 'tweet_source',
       'tweet_coord', 'tweet_place', 'numMentions', 'date', 'engagment',
       'is_reply', 'is_user_bot', 'is_outlier', 'is_valid', 'eggplant',
       'mfinger', 'curse', 'fuck', 'text_polarity', 'text_subjectivity',
       'vader_comp', 'vader_pos', 'vader_neu', 'vader_neg', 'processed_text',
       'text_token', 'text_token_rem_stop', 'text_len', 'n_mentions',
       'n_hashtags', 'n_links', 'n_emojis', 'swear_count', 'swear_present',
       'swear_severity', 'swear_rarity_by_percentage', 'censored_presence',
       'day_of_week', 'hour_of_day', 'per_engagement']

<string>:2: DtypeWarning: Columns (16,19,20) have mixed types. Specify dtype option on import or set low_memory=False.


raw processed data shape: (1116354, 54)
raw processed data columns:
Index(['user_id', 'text', 'n_retweets', 'n_favorites', 'n_user_followers',
       'n_user_following', 'n_user_posts', 'n_user_favs', 'n_user_lists',
       'user_description_text', 'user_location', 'account_age',
       'user_created_year', 'user_created_month', 'user_geo_enabled',
       'user_img_url', 'user_banner_url', 'tweet_truncated', 'tweet_source',
       'tweet_coord', 'tweet_place', 'numMentions', 'date', 'engagment',
       'is_reply', 'is_user_bot', 'is_outlier', 'is_valid', 'eggplant',
       'mfinger', 'curse', 'fuck', 'text_polarity', 'text_subjectivity',
       'vader_comp', 'vader_pos', 'vader_neu', 'vader_neg', 'processed_text',
       'text_token', 'text_token_rem_stop', 'text_len', 'n_mentions',
       'n_hashtags', 'n_links', 'n_emojis', 'swear_count', 'swear_present',
       'swear_severity', 'swear_rarity_by_percentage', 'censored_presence',
       'day_of_week', 'hour_of_day', 'per_engagement\r